In [3]:
import pandas as pd
#from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
#크롤링 대상(장소)
site = ['화양연가', '카페다디오', '이디야', '고인돌', '스타벅스', '마린씨']
#site = ['화양연가', '카페다디오', '커피나무', '고인돌', '스타벅스']
#site = ['스타벅스']
#site = ['화양연가', '마린씨']

#크롤링 후 정보를 저장할 리스트
point = []
visitor_review = []
blog_review = []
alive_data = []
remove_data = []


for i in site:
    #크롤링 할 네이버 링크
    naver_link = 'https://map.naver.com/v5/search/' + '용산 ' + i + '?c=14134149.6626597,4514872.5029643,15,0,0,0,dh&entry=plt'
    wd = webdriver.Chrome('chromedriver.exe')

    wd.get(naver_link)
    time.sleep(1)

    #네이버 map의 HTML구조를 크롤링하기 위해 frame을 나눠서 크롤링해야함.
    wd.switch_to.frame(wd.find_element_by_css_selector('iframe#searchIframe'))
    
    #네이버에 있는 두 가지 틀을 모두 접근할 수 있도록 코드 추가
    
    #일반 음식점
    frame = wd.find_elements_by_css_selector('li._22p-O._2NEjP')
    if frame:
        
        #가게명 크롤링
        spot = wd.find_elements_by_css_selector('span.place_bluelink')

        #가게명 비교(네이버에 가게가 검색되지 않을 시 remove)
        result = 0
        for j in range(len(spot)) :
            if i != spot[j].text:
                if j == len(spot)-1 :
                    remove_data.append(i)
                    result = 1
                    break
                else :
                    j = j+1
            else :
                result = 0
                store_name = spot[j].text
                break

        if result == 1:
            continue

        #주소지 크롤링
        address = wd.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[2]/div/div/span/a/span[1]')
        address_name = address.text
        
        try:
            #주소지 비교 후 장소 팝업 창으로 이동
            if address_name[3:5] == '용산':
                spot = wd.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[2]/a[1]/div/div/span[1]').click()
                time.sleep(1)

                alive_data.append(i)

                wd.switch_to.default_content()
                wd.switch_to.frame(wd.find_element_by_xpath('//*[@id="entryIframe"]'))
            else :
                remove_data.append(i)
                continue
        
        except Exception:
            wd.switch_to.default_content()
            wd.switch_to.frame(wd.find_element_by_xpath('//*[@id="entryIframe"]'))
    
            time.sleep(1)
            #가게 세부 정보 탭 바로 보이기    
            frame = wd.find_element_by_css_selector('div._2W9rG.aGSb2')
            if frame:
        
                #가게명 크롤링
                spot = wd.find_elements_by_css_selector('span._3XamX')
                alive_data.append(i)
            
    
    #프랜차이즈 음식점
    frame = wd.find_elements_by_css_selector('li._1EKsQ._12tNp')
    if frame :
        
        #가게명 크롤링
        spot = wd.find_elements_by_css_selector('span.place_bluelink')
        
    
        #용산구 내 동이름 리스트
        address_rule = ['후암동', '용산동2가', '용산동4가', '갈월동', '남영동', '동자동', '용산동1가', '서계동', '청파동1가', '청파동2가', '청파동3가', '원효로1가', '원효로2가', '문배동', '신계동', '원효로3가', '원효로4가', '신창동', '청암동', '산천동', '효창동', '용문동', '도원동', '한강로1가', '한강로2가', '한강로3가', '용산동3가', '용산동5가', '이촌동', '이태원동', '한남동', '서빙고동', '동빙고동', '주성동', '용산동6가', '보광동']
        
        #주소 크롤링
        address = wd.find_elements_by_css_selector('div._1gET-')
        
        result = 0
        result_2 = 0
        #주소지가 용산구가 맞는지 확인 후 장소 팝업 창으로 이동
        for j in address:
            address_name = j.text

            for q in address_rule:
                if address_name.find(q) != -1:
                    spot = wd.find_element_by_css_selector('span.place_bluelink').click()
                    time.sleep(1)
                    
                    alive_data.append(i)

                    wd.switch_to.default_content()
                    wd.switch_to.frame(wd.find_element_by_xpath('//*[@id="entryIframe"]'))
                    result_2 = 1
                    break
            if result_2 == 1:
                break
            else:
                result = 1
                
        
        
        if result == 1:
            remove_data.append(i)
            continue
    
        
    #별점, 방문자리뷰수, 블로그리뷰수 크롤링 후 리스트에 저장
    review = wd.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]').text
    
    if review.find('식약처') != -1:
        review = review[:review.find('식약처')-1]
    
    site_point_html = review.find('별점')
    site_visitor_review_html = review.find('방문자리뷰')
    site_blog_review_html = review.find('블로그리뷰')
    
    #별점
    if site_point_html != -1 :
        site_point = review[3:7]
        point.append(site_point)
    else :
        point.append("")
        
    #방문자리뷰
    if site_visitor_review_html != -1 :
        if site_blog_review_html != -1 : 
            site_visitor_review = review[site_visitor_review_html+6:site_blog_review_html]
            visitor_review.append(site_visitor_review)
        else :
            site_visitor_review = review[site_visitor_review_html+6:]
            visitor_review.append(site_visitor_review)
    else :
        visitor_review.append("")
        
    #블로그리뷰
    if site_blog_review_html != -1 :
        site_blog_review = review[site_blog_review_html+6:]
        blog_review.append(site_blog_review)
    else :
        blog_review.append("")

    
#리스트 출력 확인
print(point)
print(visitor_review)
print(blog_review)
print(alive_data)
print(remove_data)

['4.71', '4.44', '4.32', '4.48', '']
['50', '35', '390', '5,558', '66']
['158', '', '10', '221', '33']
['화양연가', '카페다디오', '이디야', '스타벅스', '마린씨']
['고인돌']


In [8]:
site = ['1번지남산돈까스']
df_final = []

In [9]:
for i in site:
    
        #크롤링 할 네이버 링크
        naver_link = 'https://map.naver.com/v5/search/' + '용산 ' + i + '?c=14134149.6626597,4514872.5029643,15,0,0,0,dh&entry=plt'
        wd = webdriver.Chrome('chromedriver.exe')

        wd.get(naver_link)
        time.sleep(1)

        #네이버 map의 HTML구조를 크롤링하기 위해 frame을 나눠서 크롤링해야함.
        wd.switch_to.frame(wd.find_element_by_css_selector('iframe#searchIframe'))

        #네이버에 있는 두 가지 틀을 모두 접근할 수 있도록 코드 추가

        #일반 음식점
        frame = wd.find_elements_by_css_selector('li._22p-O._2NEjP')
        if frame:

            #가게명, 주소지 크롤링
            spot = wd.find_elements_by_css_selector('span.place_bluelink._3Apve')
            address = wd.find_elements_by_css_selector('span._3hCbH')

            try:
                for j in range(len(address)):
                    address_name = address[j].text
                    #주소지 비교 후 장소 팝업 창으로 이동
                    if address_name[3:5] == '용산':
                        select = wd.find_element_by_css_selector('span.place_bluelink._3Apve').click()
                        time.sleep(1)

                        alive_data = spot[j].text

                        wd.switch_to.default_content()
                        wd.switch_to.frame(wd.find_element_by_xpath('//*[@id="entryIframe"]'))
                        break

                remove_data.append(i)
                continue

            except Exception:
                wd.switch_to.default_content()
                wd.switch_to.frame(wd.find_element_by_xpath('//*[@id="entryIframe"]'))

                time.sleep(1)
                #가게 세부 정보 탭 바로 보이기    
                frame = wd.find_element_by_css_selector('div._2W9rG.aGSb2')
                if frame:

                    #가게명 크롤링
                    spot = wd.find_element_by_css_selector('span._3XamX')
                    alive_data = spot.text


        #프랜차이즈 음식점
        frame = wd.find_elements_by_css_selector('li._1EKsQ._12tNp')
        if frame :


            #용산구 내 동이름 리스트
            address_rule = ['후암동', '용산동2가', '용산동4가', '갈월동', '남영동', '동자동', '용산동1가', '서계동', '청파동1가', '청파동2가', '청파동3가', '원효로1가', '원효로2가', '문배동', '신계동', '원효로3가', '원효로4가', '신창동', '청암동', '산천동', '효창동', '용문동', '도원동', '한강로1가', '한강로2가', '한강로3가', '용산동3가', '용산동5가', '이촌동', '이태원동', '한남동', '서빙고동', '동빙고동', '주성동', '용산동6가', '보광동']

            #가게명, 주소지 크롤링
            spot = wd.find_elements_by_css_selector('span.place_bluelink.OXiLu')
            address = wd.find_elements_by_css_selector('div._1gET-')

            result = 0
            result_2 = 0
            #주소지가 용산구가 맞는지 확인 후 장소 팝업 창으로 이동
            for j in range(len(address)):
                address_name = address[j].text

                for q in address_rule:
                    if address_name.find(q) != -1:
                        select = wd.find_element_by_css_selector('span.place_bluelink.OXiLu').click()
                        time.sleep(1)

                        alive_data = spot[j].text

                        wd.switch_to.default_content()
                        wd.switch_to.frame(wd.find_element_by_xpath('//*[@id="entryIframe"]'))
                        result_2 = 1
                        break
                if result_2 == 1:
                    break
                else:
                    result = 1



            if result == 1:
                remove_data.append(i)
                continue


        #별점, 방문자리뷰수, 블로그리뷰수 크롤링 후 리스트에 저장
        review = wd.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]').text

        if review.find('식약처') != -1:
            review = review[:review.find('식약처')-1]

        site_point_html = review.find('별점')
        site_visitor_review_html = review.find('방문자리뷰')
        site_blog_review_html = review.find('블로그리뷰')

        #별점
        if site_point_html != -1 :
            site_point = review[3:7]
            point = site_point
        else :
            point = ""

        #방문자리뷰
        if site_visitor_review_html != -1 :
            if site_blog_review_html != -1 : 
                site_visitor_review = review[site_visitor_review_html+6:site_blog_review_html]
                visitor_review = site_visitor_review
            else :
                site_visitor_review = review[site_visitor_review_html+6:]
                visitor_review = site_visitor_review
        else :
            visitor_review = ""

        #블로그리뷰
        if site_blog_review_html != -1 :
            site_blog_review = review[site_blog_review_html+6:]
            blog_review = site_blog_review
        else :
            blog_review = ""

        #방문 횟수
        #visit_count = int(df_place.loc[i]['방문 횟수'])

        #1인당 금액
        #per_money = int(df_place.loc[i]['인당 평균금액(원)'])

        #df_final
        data = [alive_data, point, visitor_review, blog_review]
        df_final.append(data)
        print(data)
        data = []

df_result = pd.DataFrame(df_final)
df_result

""
